In [ ]:
from pathlib import Path
from ebmdatalab import bq

from utils import read_sql_query
from config import DATA_DIR

PROJECT_ID = "ebmdatalab"
CREDENTIALS_PATH = 'notebooks/cred/bq-service-account.json'


In [ ]:
query = read_sql_query("andexanet.sql")
df = bq.cached_read(sql=query, csv_path=Path(DATA_DIR, "andexanet.csv"), use_cache=True)
 

In [ ]:
df.head()

In [ ]:
df["vmp_name"].unique()